In [2]:
from datasets import load_dataset

dataset = load_dataset(
    "./data/yelp_review_full")

Generating train split: 650000 examples [00:00, 981760.05 examples/s] 
Generating test split: 50000 examples [00:00, 865004.70 examples/s]


In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "/Volumes/WD_BLACK/models/bert-base-cased")

# 以最大长度进行padding，并截断


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/650000 [00:00<?, ? examples/s]


ValueError: min_length is not a valid PaddingStrategy, please select one of ['longest', 'max_length', 'do_not_pad']

In [4]:
small_train_dataset = tokenized_datasets["train"].shuffle(
    seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(
    seed=42).select(range(1000))

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("./models/bert-base-cased",
                                                           num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./models/bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import wandb
wandb.login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenize

True

In [8]:
import os
os.environ["WANDB_PROJECT"]="huggingface_wandb_test_new"

In [13]:
from transformers import TrainingArguments

# 训练超参数
model_dir = "models/bert-base-cased"
# 训练过程指标监控
# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100,
                                  # report_to="wandb",  # enable logging to W&B
                                  # # name of the W&B run (optional)
                                  # run_name="bert-base-cased",
                                  # logging_steps=1,  # how often to log to W&B
                                  )

In [14]:
import evaluate
import numpy as np
# 训练过程中的指标评估
metric = evaluate.load("./accuracy/")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
from transformers import Trainer

trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=small_train_dataset,
                  eval_dataset=small_eval_dataset,
                  compute_metrics=compute_metrics,
                  )

In [16]:
trainer.train()

ValueError: expected sequence of length 121 at dim 1 (got 139)